In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

stop_Words = text.ENGLISH_STOP_WORDS


In [4]:
df = pd.read_csv("./data/parsed/training.csv", dtype=str)
df.head()

,Unnamed: 0,Unnamed: 0.1,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,...,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Message-ID,Message-Contents,ID,Label,Scenario
0,0,0,"Fri, 15 Jun 2001 11:37:53 -0700 (PDT)",david.allan@enron.com,"david.allan@enron.com, e..carter@enron.com, el...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"Allan, David </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,401
1,1,1,"Fri, 15 Jun 2001 11:37:53 -0700 (PDT)",david.allan@enron.com,"david.allan@enron.com, e..carter@enron.com, el...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"Allan, David </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,402
2,2,2,"Fri, 15 Jun 2001 11:37:53 -0700 (PDT)",david.allan@enron.com,"david.allan@enron.com, e..carter@enron.com, el...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"Allan, David </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,403
3,3,3,"Mon, 6 Aug 2001 10:47:06 -0700 (PDT)",stacey.wales@enron.com,"stacey.wales@enron.com, e..carter@enron.com, p...",Canceled: Pulp Origination Strategy,1.0,text/plain; charset=us-ascii,7bit,"Wales, Stacey </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<19614702.1075853164028.JavaMail.evans@thyme>,"Dear Team,\n\n\nI would like to brainstorm ide...",3.438120.GTSTTLNTZ2LVIWQFALPJFBLRPP2UC0G4A,-1,401
4,4,4,"Mon, 6 Aug 2001 10:47:06 -0700 (PDT)",stacey.wales@enron.com,"stacey.wales@enron.com, e..carter@enron.com, p...",Canceled: Pulp Origination Strategy,1.0,text/plain; charset=us-ascii,7bit,"Wales, Stacey </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<19614702.1075853164028.JavaMail.evans@thyme>,"Dear Team,\n\n\nI would like to brainstorm ide...",3.438120.GTSTTLNTZ2LVIWQFALPJFBLRPP2UC0G4A,-1,402


In [6]:
import nltk, re

In [7]:
words = set(nltk.corpus.words.words())

#df["Message-Contents"].apply(len)
#df["Message-Contents"][16]
gross_email = df[df["Message-Contents"].str.contains("----")]


In [8]:
import re

'''
Email filter class that will hold all of our regexes that we can apply to ALL emails. It is helpful 
to have it as an objectbecause we can just add the regexes as instance variables and then iterate 
over all of them using <object>.__dict__.items()
'''
class Email_filter:
     def __init__(self):
        ##regexes#
        self.quoted = re.compile(r"^>(\s)*", re.MULTILINE)
        self.original_or_forward = re.compile(r"(^(\s)*-*(\s)*Original Message(\s)*-.*$)|(^____.*$)", re.MULTILINE)
        self.metadata = re.compile(r"(^From:.*$)|(^Sent:.*$)|(^To:.*$)|(^Subject:.*$)|(^Cc:.*$)|(^Date:.*$)|(^Encoding:.*$)", re.MULTILINE)
        self.ole_object = re.compile(r"(<<.*?>>)+", re.DOTALL|re.MULTILINE)
        self.smtp_header = re.compile(r"^Message-id:.*?X-Mozilla-Status.*?$", re.DOTALL|re.MULTILINE)
        self.received = re.compile(r"^Received:(.*?)\([A-Z]{3}\)", re.DOTALL|re.MULTILINE)

'''
Method to actually filter the emails - creates an email filter object and loops through all of the 
regexes it contains, running each of them on an email. If the email is a reply, it will first 
cut out all of the replies.
'''
def filter_email(s):
    if type(s) != str:
        return("")
    e = Email_filter()
    ret = s
    for variable_name, regex in e.__dict__.items():  
        ret = re.sub(regex, "", ret)
    
    return ret

'''
Method to cut out all of the replies of emails
'''
def filter_reply(s):
    replies = re.compile(r"(^(\s)*-*(\s)*Original Message.*)", re.MULTILINE|re.DOTALL)
    ret = re.sub(replies, "", s)
    return ret


'''
metadata parsing
'''
from datetime import datetime
import re
def parse_date(d):
    d = d.replace(',', '')
    redundancy_filter = d.split(' (')
    string = redundancy_filter[0]
    datetime_object = datetime.strptime(string, '%a %d %b %Y %X %z')
    return datetime_object


def x_strip(s):
    if s != "":
        array = re.split(r'</O.*?\>', s)
        array = [x.strip() for x in array]
        return array
    else:
        return([])

def to_from_strip(s):
    if s != "":
        return s.split(',')
    else:
        return([])
    
def parse_metadata(df):
    df['Date'] = df['Date'].apply(parse_date)
    df['To'] = df['To'].apply(to_from_strip)
    df['From'] = df['From'].apply(to_from_strip)
    df['X-To'] = df['X-To'].apply(x_strip)
    df['X-From'] = df['X-From'].apply(x_strip)
    
    return(df)
    
def full_filter_email(df):
    df = df.replace(np.nan, '', regex=True)
    df.loc[df["Subject"].str.lower().str.startswith('re')]["Message-Contents"].apply(filter_reply)
    df["Message-Contents"] = df["Message-Contents"].apply(filter_email)
    df = parse_metadata(df)
    return(df)

In [9]:
gross_email_filtered = pd.DataFrame()
gross_email_filtered = full_filter_email(df)

In [10]:
gross_email_filtered.head()

,Unnamed: 0,Unnamed: 0.1,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,...,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Message-ID,Message-Contents,ID,Label,Scenario
0,0,0,2001-06-15 11:37:53-07:00,[david.allan@enron.com],"[david.allan@enron.com, e..carter@enron.com, ...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"[Allan, David, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,401
1,1,1,2001-06-15 11:37:53-07:00,[david.allan@enron.com],"[david.allan@enron.com, e..carter@enron.com, ...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"[Allan, David, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,402
2,2,2,2001-06-15 11:37:53-07:00,[david.allan@enron.com],"[david.allan@enron.com, e..carter@enron.com, ...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"[Allan, David, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,403
3,3,3,2001-08-06 10:47:06-07:00,[stacey.wales@enron.com],"[stacey.wales@enron.com, e..carter@enron.com,...",Canceled: Pulp Origination Strategy,1.0,text/plain; charset=us-ascii,7bit,"[Wales, Stacey, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<19614702.1075853164028.JavaMail.evans@thyme>,"Dear Team,\n\n\nI would like to brainstorm ide...",3.438120.GTSTTLNTZ2LVIWQFALPJFBLRPP2UC0G4A,-1,401
4,4,4,2001-08-06 10:47:06-07:00,[stacey.wales@enron.com],"[stacey.wales@enron.com, e..carter@enron.com,...",Canceled: Pulp Origination Strategy,1.0,text/plain; charset=us-ascii,7bit,"[Wales, Stacey, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<19614702.1075853164028.JavaMail.evans@thyme>,"Dear Team,\n\n\nI would like to brainstorm ide...",3.438120.GTSTTLNTZ2LVIWQFALPJFBLRPP2UC0G4A,-1,402


In [11]:
gross_email_filtered.to_pickle('pickled_data_test.pickle')

KeyboardInterrupt: 

In [13]:
print(gross_email["Subject"][50846])
print(filter_email(gross_email["Message-Contents"][50846], is_reply=False))

RE: Transfer of Dana Davis


TypeError: filter_email() got an unexpected keyword argument 'is_reply'

In [16]:
print(gross_email["Message-Contents"][50846])
print(gross_email_filtered["Message-Contents"][50846])

Please check with Maria Lebeau who has been coordinating hiring in the GCP group.

 -----Original Message-----
From: 	McLoughlin, Hector  
Sent:	Friday, November 02, 2001 2:20 PM
To:	Boals, Adrial; Davis, Dana; Scribner, James; Carrizales, Blanca
Cc:	Campos, Karen E.; Ormston, Kevin; Perkins, Ramona
Subject:	Transfer of Dana Davis

I'm seeking some clarification about the possible transfer of Dana Davis into James Scribner's cost center.    It is my understanding that she was to transfer effective 11/1/01?    Was the position by any chance posted?  Do we need to create a position or is it a transfer into an existing position?
Thanks for your help, 
h
Please check with Maria Lebeau who has been coordinating hiring in the GCP group.







I'm seeking some clarification about the possible transfer of Dana Davis into James Scribner's cost center.    It is my understanding that she was to transfer effective 11/1/01?    Was the position by any chance posted?  Do we need to create a position

## TF-IDF

In [3]:
import time
emails = pd.read_pickle("data/parsed/pickles/pickled_data_test.pickle")

start_time = time.time()
vectorizer = TfidfVectorizer(stop_words = stop_Words, min_df = .0001)
vectorized = vectorizer.fit_transform(emails["Message-Contents"])
print(time.time() - start_time, 's')
start_time1 = time.time()
dumb_numbers = [s for s in vectorizer.get_feature_names() if (("0" in s) or ("1" in s) or ("2" in s) or ("3" in s) or ("4" in s) or ("5" in s) or ("6" in s) or ("7" in s) or ("8" in s) or ("9" in s) or ("_" in s))]
stop_words = text.ENGLISH_STOP_WORDS.union(dumb_numbers)
print(time.time() - start_time1, 's')
start_time2 = time.time()
vectorizer = TfidfVectorizer(stop_words = stop_words, min_df = .0001)
vectorized = vectorizer.fit_transform(emails["Message-Contents"])
print(time.time() - start_time2, 's')

#print(vectorized.shape)
#print(vectorized)
#print(vectorized.toarray())
#print(vectorizer.idf_)
#print(vectorizer.get_feature_names()[:20])
#
#for item in vectorized:
#   print(item)
#   for j in range(len(vectorizer.get_feature_names())):
#       print(vectorizer.get_feature_names()[j], vectorized.toarray()[i][j])

42.93179440498352 s
0.03823590278625488 s
40.97611904144287 s


In [25]:
emails_short = emails[:2000]
vectorized_short = vectorizer.fit_transform(emails_short["Message-Contents"])
vectorized_short_T = vectorized_short.T
print(vectorized_short.shape)

(2000, 8016)


In [ ]:
print(vectorized.shape)
print(vectorizer.get_feature_names()[:1000])

## LSA
We take the tf-idf matrix and run TruncatedSVD on it with whatever number of dimensions we want to reduce it to. The issue is I don't believe I can pull the categories. I can pull the highest ranking words from each category, but there is no storage of the category iteself. Currently a WIP on pulling each category, but for right now I'm doing the dot product so I can map term to term.

In [28]:
# Implemented without stemming to better work with current tf-idf code and the nasty documents

svd = TruncatedSVD(n_components = 100, algorithm='randomized')
lsa = svd.fit_transform(vectorized_short.T)
documentLSA = svd.fit_transform(vectorized_short_T.T)

print(lsa)

def get_closest(term, vectorizer ,model):
 
    index = vectorizer.vocabulary_[term]

    model = np.dot(model, model.T)
    search_space = np.concatenate( (model[index][:index] , model[index][(index+1):]) )  
    out = np.argmax(search_space)
    if out<index:
        return vectorizer.get_feature_names()[out]
    else:
        return vectorizer.get_feature_names()[(out+1)]
    
def get_closest_document(index, vectorizer, model):
    model = np.dot(model, model.T)
    search_space = np.concatenate( (model[index][:index] , model[index][(index+1):]) )  
    out = np.argmax(search_space)
    if out<index:
        return vectorizer.get_feature_names()[out]
    else:
        return vectorizer.get_feature_names()[(out+1)]
    
def get_k_closest(k,term,vectorizer,model):
    index = vectorizer.vocabulary_[term]
    print("index: ", index)

    # This is one approach where we take the dot product
    # This results in a mapping of terms to terms based on similarity to each other
    model = np.dot(model,model.T)
    print(model.shape)

    closest_terms = {}
    print(len(model))
    for i in range(len(model)):
        closest_terms[vectorizer.get_feature_names()[i]] = model[index][i]
    #print(closest_terms)
    sorted_list = sorted(closest_terms , key = lambda l : closest_terms[l])
    return sorted_list[::-1][0:k]

# WIP
def get_topics(transformer, model): 
    model = np.dot(model,model.T)
    
    closest_terms = {}
    for i in range(len(model)):
        for i in range(len(model)):
            closest_terms[vectorizer.get_feature_names()[i]] = model[index][i]
            

    


[[  5.03382899e-04   6.72647242e-04   7.69613596e-04 ...,  -1.89123644e-03
   -1.66487698e-04   1.11538421e-03]
 [  2.90729375e-02  -2.74823200e-03  -4.14041714e-03 ...,  -9.70413021e-03
    2.51167466e-02   6.88714386e-03]
 [  1.60719774e-02  -9.73367449e-04   1.61678035e-02 ...,   1.03743487e-02
   -3.63611745e-03  -2.41640806e-02]
 ..., 
 [  6.87105181e-03   1.33535357e-05   4.32844390e-03 ...,   2.58238905e-05
    2.59665463e-03   2.73669122e-03]
 [  2.17029546e-03   1.17587721e-03   3.64371737e-03 ...,  -1.20280565e-04
   -5.93490854e-05   4.13429758e-05]
 [  1.53171308e-04  -1.80402530e-05  -7.49766564e-05 ...,  -7.75598521e-04
    1.37914787e-05   1.32872093e-05]]


In [17]:
print(get_closest("stock", vectorizer, lsa))
print(get_k_closest(5, "stock", vectorizer, lsa))

price
index:  6870
(8016, 8016)
8016
['price', 'tonnes', 'help', 'thanks', 'sappi']


In [47]:
print(get_closest_document(1500, vectorizer, documentLSA))

# Replace DOCUMENT NUMBER with a document that you want to find similar for

calculator


In [ ]:
# WIP
print(get_topics(vectorizer, lsa))